In [6]:
import sqlite3
import pandas as pd

from contextlib import closing

In [7]:
db_file = '/clara/testData/chemInformatics/data/db/chembl_27.db'
chembl_db = 'file://%s?mode=ro' % db_file

In [8]:
select_stmt = '''
    SELECT cp.molregno,
    cs.canonical_smiles,
    di.max_phase_for_ind,
    cp.* 
FROM
    drug_indication AS di,
    compound_structures AS cs,
    compound_properties AS cp
WHERE
	di.molregno = cs.molregno
	AND di.molregno = cp.molregno
    AND di.max_phase_for_ind >= 3
    AND canonical_smiles is not null
GROUP BY cp.molregno,
    cs.canonical_smiles
HAVING di.max_phase_for_ind = max(di.max_phase_for_ind);
'''

In [9]:
with closing(sqlite3.connect(chembl_db, uri=True)) as con:
    df = pd.read_sql(select_stmt, con)
    
df.shape

(2768, 26)

In [15]:
df

,molregno,canonical_smiles,max_phase_for_ind,molregno,mw_freebase,alogp,hba,hbd,psa,rtb,...,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations
0,97,COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,4,97,383.41,1.78,8.0,1.0,106.95,4.0,...,NEUTRAL,383.41,3.0,28.0,0.73,383.1594,C19H21N5O4,9.0,2.0,0.0
1,115,CN1CCC[C@H]1c1cccnc1,4,115,162.24,1.85,2.0,0.0,16.13,1.0,...,BASE,162.24,1.0,12.0,0.63,162.1157,C10H14N2,2.0,0.0,0.0
2,146,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23,4,146,361.37,1.54,6.0,1.0,75.01,2.0,...,ACID,361.37,2.0,26.0,0.87,361.1438,C18H20FN3O4,7.0,1.0,0.0
3,154,CCN(CC)CCN(Cc1ccc(-c2ccc(C(F)(F)F)cc2)cc1)C(=O...,3,154,666.79,7.22,6.0,0.0,58.44,13.0,...,NEUTRAL,666.79,4.0,47.0,0.09,666.2652,C36H38F4N4O2S,6.0,0.0,2.0
4,173,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1,4,173,357.79,3.93,4.0,1.0,68.53,4.0,...,ACID,357.79,3.0,25.0,0.77,357.0768,C19H16ClNO4,5.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,2335634,Cn1c(=O)c2c(ncn2CC(=O)O)n(C)c1=O.Nc1c(Br)cc(Br...,3,2335634,616.31,NaN,NaN,NaN,NaN,NaN,...,None,616.31,NaN,NaN,NaN,614.0488,C22H28Br2N6O5,NaN,NaN,NaN
2764,2335820,C[C@@H]1CN(c2nc(-n3ccc(OCCC4(C(F)(F)F)CC4)n3)c...,3,2335820,591.66,5.12,8.0,1.0,106.42,9.0,...,ACID,591.66,3.0,41.0,0.37,591.2127,C28H32F3N5O4S,9.0,1.0,2.0
2765,2335836,C=CC(=O)Nc1cccc(Oc2nc(Nc3ccc(N4CCN(C)CC4)c(F)c...,3,2335836,487.54,4.51,7.0,3.0,98.41,7.0,...,NEUTRAL,487.54,4.0,36.0,0.33,487.2132,C26H26FN7O2,9.0,3.0,0.0
2766,2336099,Cc1nn(C)cc1S(=O)(=O)NC(=O)c1ccc(-n2ccc(OCC(C)(...,3,2336099,597.66,4.02,10.0,1.0,124.24,8.0,...,ACID,597.66,3.0,41.0,0.41,597.2345,C26H34F3N7O4S,11.0,1.0,2.0


In [24]:
set(df.columns.tolist()) - set(df.columns[df.isna().any()].tolist())

{'canonical_smiles',
 'full_molformula',
 'full_mwt',
 'max_phase_for_ind',
 'molregno',
 'mw_freebase',
 'mw_monoisotopic'}